## Data Loading

In [1]:
import h5py
import numpy as np
import pickle

# data = h5py.File('/home/sunji/ANN/glove_200_angular/glove-200-angular.hdf5', 'r')
# data_train = np.array(data['train'])
# data_test = np.array(data['test'])
# with open('/home/sunji/ANN/glove_200_angular/clusters_glove_200_angular.pkl', 'rb') as f:
#     clusters = pickle.load(f)
# with open('/home/sunji/ANN/glove_200_angular/ground_truth_glove_200_angular_0_4_0_5.pkl', 'rb') as f:
#     ground_truth_total = pickle.load(f)
    
# data = h5py.File('/home/sunji/ANN/fashion_mnist_784_euclidean/fashion-mnist-784-euclidean.hdf5', 'r')
# data_train = np.array(data['train'])
# data_test = np.array(data['test'])
# with open('/home/sunji/ANN/fashion_mnist_784_euclidean/clusters_fashion_mnist_784_euclidean.pkl', 'rb') as f:
#     clusters = pickle.load(f)
# with open('/home/sunji/ANN/fashion_mnist_784_euclidean/ground_truth_fashion_mnist_784_euclidean_0_0_0_5.pkl', 'rb') as f:
#     ground_truth_total = pickle.load(f)

# data = h5py.File('/home/sunji/ANN/nytimes_256_angular/nytimes-256-angular.hdf5', 'r')
# data_train = np.array(data['train'])
# data_test = np.array(data['test'])
# with open('/home/sunji/ANN/nytimes_256_angular/clusters_nytimes_256_angular.pkl', 'rb') as f:
#     clusters = pickle.load(f)
# with open('/home/sunji/ANN/nytimes_256_angular/ground_truth_nytimes_256_angular_0_4_0_5.pkl', 'rb') as f:
#     ground_truth_total = pickle.load(f)

# data = h5py.File('/home/sunji/ANN/sift_128_euclidean/sift-128-euclidean.hdf5', 'r')
# data_train = np.array(data['train'])
# data_test = np.array(data['test'])
# with open('/home/sunji/ANN/sift_128_euclidean/clusters_sift_128_euclidean.pkl', 'rb') as f:
#     clusters = pickle.load(f)
# with open('/home/sunji/ANN/sift_128_euclidean/ground_truth_sift_128_euclidean_0_0_0_2.pkl', 'rb') as f:
#     ground_truth_total = pickle.load(f)
    
# data = h5py.File('/home/sunji/ANN/kosarak_jaccard/kosarak-jaccard.hdf5', 'r')
# data_train = np.array(data['train'])
# data_test = np.array(data['test'])
# with open('/home/sunji/ANN/kosarak_jaccard/clusters_kosarak_jaccard.pkl', 'rb') as f:
#     clusters = pickle.load(f)
# with open('/home/sunji/ANN/kosarak_jaccard/ground_truth_kosarak_jaccard_0_9_1_0.pkl', 'rb') as f:
#     ground_truth_total = pickle.load(f)

data = h5py.File('/home/sunji/ANN/gist_960_euclidean/gist-960-euclidean.hdf5', 'r')
data_train = np.array(data['train'])
data_test = np.array(data['test'])
with open('/home/sunji/ANN/gist_960_euclidean/clusters_gist_960_euclidean.pkl', 'rb') as f:
    clusters = pickle.load(f)
with open('/home/sunji/ANN/gist_960_euclidean/ground_truth_gist_960_euclidean_0_0_0_1.pkl', 'rb') as f:
    ground_truth_total = pickle.load(f)


In [2]:
cluster_size = len(clusters)

In [3]:
ground_truth_total_level = [[[] for _ in range(1000)] for _ in range(cluster_size)]
for clus in range(cluster_size):
    for t in ground_truth_total[clus]:
        ground_truth_total_level[t[0]][t[1]].append(t)

In [4]:
centroids = []
for cluster in clusters:
    centroids.append(np.mean(cluster))

## Prepare Inputs

In [5]:
from numpy import dot
from numpy.linalg import norm
from scipy import spatial

def euclidean_dist_normalized(x1, x2=None, eps=1e-8):
    if np.isnan(x2):
        return 1.0
    left = x1
    right = x2
    return np.sqrt(((left - right) ** 2).mean())

def angular_dist(x1, x2=None, eps=1e-8):
    cosine_sim = 1 - spatial.distance.cosine(x1, x2)
#     print (cosine_sim)
    distance = np.arccos(cosine_sim) / 3.14159267
    return distance

train_features = []
train_thresholds = []
train_targets = []
train_cards = []
slot = 0.001
for train_id in range(800):
    cardinality = [0 for _ in range(cluster_size)]
    query_ids = np.random.choice(800, 30, replace=False)
#     queries = []
#     for query_id in query_ids:
#         queries.append(data_test[query_id])
    for threshold_id, threshold in enumerate(np.arange(0.0, 0.1, slot)):
        indicator = []
        cards = []
        for cluster_id in range(cluster_size):
            for query_id in query_ids:
                cardinality[cluster_id] += ground_truth_total_level[cluster_id][query_id][threshold_id][-1]
            if cardinality[cluster_id] > 0:
                indicator.append(1)
            else:
                indicator.append(0)
            cards.append(cardinality[cluster_id])
        train_features.append(query_ids)
        train_thresholds.append([threshold+slot])
        train_targets.append(indicator)
        train_cards.append(cards)

test_features = []
test_thresholds = []
test_targets = []
test_cards = []
for test_id in range(200):
    cardinality = [0 for _ in range(cluster_size)]
    query_ids = np.random.choice(200, 30, replace=False) + 800
#     queries = []
#     for query_id in query_ids:
#         queries.append(data_test[query_id])
    for threshold_id, threshold in enumerate(np.arange(0.0, 0.1, slot)):
        indicator = []
        cards = []
        for cluster_id in range(cluster_size):
            for query_id in query_ids:
                cardinality[cluster_id] += ground_truth_total_level[cluster_id][query_id][threshold_id][-1]
            if cardinality[cluster_id] > 0:
                indicator.append(1)
            else:
                indicator.append(0)
            cards.append(cardinality[cluster_id])
        test_features.append(query_ids)
        test_thresholds.append([threshold+slot])
        test_targets.append(indicator)
        test_cards.append(cards)
        
        

In [14]:
for i in range(3):
    print (np.random.choice(10000, 30, replace=False))

[8767  616 5086 3584 7169 9913 1754  753 2437 8300 6602 2495 7491 6415
 7751 6538 2632 9530 6118 5148 2524 3453 1302  420 2507  276 2582 1841
 5304 8075]
[2904 9979 1583 7084 7576 1973 7912 3251 1198  117 1745 3507 7229 1735
 3633 1291 4440 2013 2226 4076 6892 9597  258 1422 1700 2703 1742 2114
 5352 8895]
[5387 6360 9812 6131  449 9745 8533  988 7473  210 3270 4829 6187 4762
 9859 6138 7068 4879 5949 1850 7460 8457 3986  414 8320 3283 4815 7148
 5698 2288]


In [6]:
import torch
import torch.utils.data

batch_size = 128
train_loader = torch.utils.data.DataLoader(
        torch.utils.data.TensorDataset(torch.FloatTensor(train_features), torch.FloatTensor(train_thresholds), torch.FloatTensor(train_targets), torch.FloatTensor(train_cards)), batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(
        torch.utils.data.TensorDataset(torch.FloatTensor(test_features), torch.FloatTensor(test_thresholds), torch.FloatTensor(test_targets), torch.FloatTensor(test_cards)), batch_size=batch_size, shuffle=True)


## Multi-label Networks

In [7]:
from __future__ import print_function
import argparse
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

input_dimension = 960
cluster_dimension = cluster_size
hidden_num = 256
output_num = cluster_size

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        
        self.emb_nn1 = nn.Linear(input_dimension, hidden_num)
        self.emb_nn2 = nn.Linear(hidden_num, input_dimension)
        
        self.nn1 = nn.Linear(input_dimension, hidden_num)
        self.nn2 = nn.Linear(hidden_num, hidden_num)
#         self.nn3 = nn.Linear(hidden_num, hidden_num)
        
#         self.dist1 = nn.Linear(cluster_dimension, hidden_num)
#         self.dist2 = nn.Linear(hidden_num, hidden_num)
        
#         self.nn4 = nn.Linear(hidden_num, hidden_num)
        self.nn5 = nn.Linear(hidden_num, output_num)
        
        self.thres1 = nn.Linear(1, hidden_num)
        self.thres2 = nn.Linear(hidden_num, 1)

    def forward(self, queries, thresholds):
        batch_size = queries.shape[0]
        set_size = queries.shape[1]
        queries = queries.view(batch_size * set_size, -1)
        
        queries = F.relu(self.emb_nn1(queries))
        queries = F.relu(self.emb_nn2(queries))
        queries = queries.view(batch_size, set_size, -1).mean(dim=1)
        
        out1 = F.relu(self.nn1(queries))
        out2 = F.relu(self.nn2(out1))
#         out3 = F.relu(self.nn3(out2))
#         print (distances.shape)
#         distance1 = F.relu(self.dist1(distances))
#         distance2 = F.relu(self.dist2(distance1))
        
        thresholds_1 = F.relu(self.thres1(thresholds))
        thresholds_2 = self.thres2(thresholds_1)

#         out4 = F.relu(self.nn4((out2 + distance2) / 2))
        out5 = self.nn5(out2)
        
        probability = F.sigmoid(out5 + thresholds_2)
        return probability

def loss_fn(estimates, targets, cards):
    punish_idx = (estimates < 0.5).float()
    return F.mse_loss(estimates, targets) + 0.02 * torch.log(((0.5 - estimates) * cards * punish_idx).mean() + 1.0)

def print_loss(estimates, targets, cards):
    true_positive = 0.0
    true_negative = 0.0
    false_positive = 0.0
    false_negative = 0.0
    num_elements = estimates.shape[1]
    for est, tar in zip(estimates, targets):
        for i in range(num_elements):
            if est[i] < 0.5 and tar[i] == 0:
                true_negative += 1
            elif est[i] < 0.5 and tar[i] == 1:
                false_negative += 1
            elif est[i] >= 0.5 and tar[i] == 0:
                false_positive += 1
            else:
                true_positive += 1
    precision = true_positive / (true_positive + false_positive)
    recall = true_positive / (true_positive + false_negative)
    total_card = cards.sum(dim=1)
#     print ('total_card: ', total_card.shape)
    miss_card = torch.FloatTensor([cards[i][((estimates[i] < 0.5).nonzero())].sum() for i in range(cards.shape[0])])
#     print ('miss_card: ', miss_card.shape)
    miss_rate = (miss_card / (total_card + 0.1)).mean()
    return precision, recall, miss_rate

In [8]:
model = Model()
opt = optim.Adam(model.parameters(), lr=0.001)
for e in range(5):
    model.train()
    for batch_idx, (features, thresholds, targets, cards) in enumerate(train_loader):
#         print (features)
        queries = torch.FloatTensor(data_test[features.type(torch.LongTensor)].astype(float))
        x = Variable(queries)
        y = Variable(targets.unsqueeze(1))
        z = Variable(thresholds)
        opt.zero_grad()
        estimates = model(x, z)
        loss = loss_fn(estimates, targets, cards)
        if batch_idx % 100 == 0:
            print('Training: Iteration {0}, Batch {1}, Loss {2}'.format(e, batch_idx, loss.item()))
        loss.backward()
        opt.step()
        next(model.thres1.parameters()).data.clamp_(0)
        next(model.thres2.parameters()).data.clamp_(0)

    model.eval()
    test_loss = 0.0
    precision = 0.0
    recall = 0.0
    miss_rate = 0.0
    for batch_idx, (features, thresholds, targets, cards) in enumerate(test_loader):
        queries = torch.FloatTensor(data_test[features.type(torch.LongTensor)].astype(float))
        x = Variable(queries)
        y = Variable(targets.unsqueeze(1))
        z = Variable(thresholds)
        estimates = model(x, z)
        loss = loss_fn(estimates, targets, cards)
        test_loss += loss.item()
        prec, rec, miss = print_loss(estimates, targets, cards)
        precision += prec
        recall += rec
        miss_rate += miss
        if batch_idx % 100 == 0:
            print ('Testing: Iteration {0}, Batch {1}, Loss {2}, Precision {3}, Recall {4}, Miss {5}'.format(e, batch_idx, loss.item(), prec, rec, miss))
    test_loss /= len(test_loader)
    precision /= len(test_loader)
    recall /= len(test_loader)
    miss_rate /= len(test_loader)
    print ('Testing: Loss {0}, Precision {1}, Recall {2}, Miss {3}'.format(test_loss, precision, recall, miss_rate))
    
    

/home/sunji/anaconda3/lib/python3.7/site-packages/torch/nn/functional.py:1332: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Training: Iteration 0, Batch 0, Loss 0.40437230467796326
Training: Iteration 0, Batch 100, Loss 0.22659610211849213
Training: Iteration 0, Batch 200, Loss 0.18407416343688965
Training: Iteration 0, Batch 300, Loss 0.20684599876403809
Training: Iteration 0, Batch 400, Loss 0.20094846189022064
Training: Iteration 0, Batch 500, Loss 0.1804567128419876
Training: Iteration 0, Batch 600, Loss 0.15404315292835236
Testing: Iteration 0, Batch 0, Loss 0.1489168107509613, Precision 0.7724590163934426, Recall 0.9961343319642426, Miss 0.015953045338392258
Testing: Iteration 0, Batch 100, Loss 0.146820530295372, Precision 0.7854949421965318, Recall 0.9958777052559258, Miss 5.017125204176409e-06
Testing: Loss 0.1545295648893733, Precision 0.7533175598102643, Recall 0.9949581612236158, Miss 0.012418574653565884
Training: Iteration 1, Batch 0, Loss 0.1505347341299057
Training: Iteration 1, Batch 100, Loss 0.13088864088058472
Training: Iteration 1, Batch 200, Loss 0.11520937830209732
Training: Iteration

In [ ]:
(distances[2], features[2])

In [ ]:
for name, param in model.thres1.named_parameters():
    print (name, param.data)

In [9]:
torch.save(model.state_dict(), '/home/sunji/ANN/join/gist_960_euclidean/global_gist_960_euclidean_punish_query_threshold_monotonic.model')


## Model Usage

In [10]:
model = Model()
model.load_state_dict(torch.load('/home/sunji/ANN/sift_128_euclidean/global_sift_128_euclidean_binary_query_threshold.model'))
model.eval()

FileNotFoundError: [Errno 2] No such file or directory: '/home/sunji/ANN/sift_128_euclidean/global_sift_128_euclidean_binary_query_threshold.model'

In [39]:
for batch_idx, (features, thresholds, distances, targets, cards) in enumerate(test_loader):
    x = Variable(features)
    y = Variable(targets.unsqueeze(1))
    z = Variable(thresholds)
    dist = Variable(distances)
    estimates = model(x, dist, z)
    loss = loss_fn(estimates, targets, cards)
    prec, rec, miss_rate = print_loss(estimates, targets, cards)
    print (loss.item(), prec, rec, miss_rate)
#     print (targets[0])

total_card:  torch.Size([128])
miss_card:  torch.Size([128])
0.011616033501923084 0.9865168539325843 0.9653655854865311 tensor(0.0307)
total_card:  torch.Size([128])
miss_card:  torch.Size([128])
0.013758893124759197 0.9868929952728835 0.9659305993690852 tensor(0.0149)
total_card:  torch.Size([128])
miss_card:  torch.Size([128])
0.010944121517241001 0.9889834752128193 0.9657701711491442 tensor(0.0170)
total_card:  torch.Size([128])
miss_card:  torch.Size([128])
0.012130827642977238 0.9878915504080021 0.9596011250319612 tensor(0.0179)
total_card:  torch.Size([128])
miss_card:  torch.Size([128])
0.011950699612498283 0.9855285749325484 0.972645848462842 tensor(0.0386)
total_card:  torch.Size([128])
miss_card:  torch.Size([128])
0.014175038784742355 0.9893148962916405 0.9622987568779295 tensor(0.0226)
total_card:  torch.Size([128])
miss_card:  torch.Size([128])
0.014790528453886509 0.9867578900904878 0.9652417962003454 tensor(0.0361)
total_card:  torch.Size([128])
miss_card:  torch.Size([1

KeyboardInterrupt: 

In [ ]:
use_cuda = torch.cuda.is_available()

#     torch.manual_seed(args.seed)

device = torch.device("cuda" if use_cuda else "cpu")
train_dataset = np.array(f['train'])
test_dataset = np.array(f['test'])
train_lefts, train_rights, test_lefts, test_rights = prepare_dataset(train_dataset, test_dataset, train_num, test_num)

train_loader = torch.utils.data.DataLoader(
    (train_lefts, train_rights), batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    (test_lefts, test_rights), batch_size=batch_size, shuffle=True)


In [ ]:
# hash_distances, input_distances = test(model, device, train_loader)
hash_distances, input_distances = test(model, device, test_loader)

In [ ]:
lefts = torch.FloatTensor([f['train'][0] for x in range(999)])
rights = torch.FloatTensor(f['train'][1:1000])
inputdistance = angular_distance(lefts, rights).detach().numpy()
hashdistance = l1_distance(model(lefts), model(rights)).detach().numpy()


In [ ]:
# for xx in zip(inputdistance, hashdistance):
#     print (xx[0], xx[1])
index_1 = np.argsort(hashdistance, 0)
index_2 = np.argsort(inputdistance, 0)
# np.random.shuffle(index_2)

input_index = {}
for pos, idx in enumerate(index_2):
    input_index[idx] = pos
sum = 0.0
for pos, idx in enumerate(index_1):
    sum += np.abs(pos - input_index[idx])
sum / len(index_1)

In [ ]:
xxx = np.sort(inputdistance, 0)
plt.plot(xxx)
plt.show()

In [ ]:
import math
distances = []
for i in index_1:
    distances.append(math.floor(inputdistance[i].item()* 40))

In [ ]:
import matplotlib.pyplot as plt

plt.plot(distances)
plt.show()

In [ ]:
F.cosine_similarity(torch.FloatTensor(f['train'][0]).unsqueeze(0), torch.FloatTensor(f['train'][6]).unsqueeze(0), dim=1, eps=1e-8)

In [ ]:
for x, y in zip(hash_distances[0][0:30], input_distances[0][0:30]):
    print (x, y)

In [ ]:
dataset_vector = model(torch.FloatTensor(f['train']))

In [ ]:
query_vector = model(torch.FloatTensor(f['test']))

In [ ]:
def binarization(vector):
    query_codes = []
    for v in vector:
        binary_code = []
        for e in v:
            if e < 0.5:
                binary_code.append(0)
            else:
                binary_code.append(1)
        query_codes.append(binary_code)
    return np.array(query_codes)
dataset_binary = binarization(dataset_vector.detach().numpy())
query_binary = binarization(query_vector.detach().numpy())

In [ ]:
len(dataset_binary)

In [ ]:
len(query_binary)

In [ ]:
import math
hash_table = {}
for idx, point in enumerate(dataset_binary):
    pos = 0
    key = 0
    for d in point:
        key += d * math.pow(2, pos)
        pos += 1
    if key in hash_table:
        hash_table[key].append(idx)
    else:
        hash_table[key] = [idx]

In [ ]:
f['neighbors'][:]

In [ ]:
def find_candidate_distance(vector, hash_table, candidate_num):
    candidate = []
    for point in query_binary:
        cand = []
        dis = 0
        while len(cand) < 100:
            pos = 0
            key = 0
            for d in point:
                key += d * math.pow(2, pos)
                pos += 1
            if key in hash_table:
                candidate.append(hash_table[key])
    return candidate
find_candidate_0_distance(query_binary, hash_table)

In [ ]:
class Node(object):
    def __init__(hash_code, data_index_set):
        self.hash_code = hash_code
        self.data_index_set = data_index_set
        self.children = []
        
    def isLeaf():
        return len(self.children) == 0
    
    def train(dataset):
        train_data = dataset[self.data_index_set]
        self.model = train(dataset)
        
    def partition():
        points = dataset[self.data_index_set]
        hash_table = {}
        codes = self.model(points)
        for idx, code in enumerate(codes):
            if code in hash_table:
                hash_table[code].append(self.data_index_set[idx])
            else:
                hash_table[code] = [self.data_index_set[idx]]
        for key,value in d.items():
            self.children.append(Node(key, value))
    
    def search(query, dataset):
        if self.isLeaf():
            return validate(dataset[self.data_index_set])
        else:
            children_idxes = select_children(query)
            result = []
            for idx in children_idxes:
                result += self.children[idx].search(query, dataset)
            return result
    
    
    

def index_construction(dataset):
    model = train(dataset)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import manifold, datasets

data = np.array(f['train'])

tsne = manifold.TSNE(n_components=2, init='pca', random_state=501)
X_tsne = tsne.fit_transform(data[np.random.choice(data.shape[0], 100000, replace=False)])